# Régression Linéaire pour la prédiction des prix de voiture

Modèle simple pour prédire les prix en fonction du kilométrage.

## Définitions :

- **Normalisation** : Redimensionner les variables numériques pour qu'elles soient comparables sur une échelle commune.
- **Machine Learning** : Donner à une machine la capacité d'apprendre sans la programmer de façon explicite.
- **Apprentissage supervisé** : Technique d'apprentissage la plus courante en machine learning. On donne des exemples à la machine qu'elle doit étudier pour en créer des modèles.
- **Dataset** : Ensemble d'exemples données à la machine pour apprendre (tableau de données).
- **Problème de regression** : On cherche à prédire la valeur d'une variable continue, c'est-à-dire une variable qui peux prendre une infinité de valeurs.
- **Problème de classification** : On cherche à prédire la valeur d'une variable discrète, c'est-à-dire une variable qui prends certaines valeurs.

## Prérequis :

Pour travailler de manière plus optimale, l'utilisation de matrice est fortement recommandé. Pour ce projet, j'ai décidé de créer moi-même une classe `Matrix` contenant toutes les opérations nécessaires pour la régression linéaire. Il est en revenche recommandé d'utiliser des librairies externes, tel que `numpy` ou `pandas` pour plus de facilité.

In [14]:
class Matrix(object):
    values: list[list[float]]

    def __init__(self, values: list[list[float]]):
        self.values = values

    def shape(self) -> tuple[int, int]:
        return len(self.values), len(self.values[0])

    def min(self) -> float:
        return min(min(value) for value in self.values)

    def max(self) -> float:
        return max(max(value) for value in self.values)

    def sum(self) -> float:
        return sum(sum(row) for row in self.values)

    def substract(self, matrix: list[list[float]]) -> list[list[float]]:
        return [[i - j for i, j in zip(*m)] for m in zip(self.values, matrix)]

    def multiply(self, matrix: list[list[float]]) -> list[list[float]]:
        if len(self.values[0]) != len(matrix):
            raise ValueError("The number of columns in the first matrix must correspond to the number of rows in the second matrix.")
        return [[sum(m * n for m, n in zip(i, j)) for j in zip(*matrix)] for i in self.values]

    def transpose(self) -> list[list[float]]:
        return list(map(list, zip(*self.values)))

    def scalar(self, n: float) -> list[list[float]]:
        return [[x * n for x in row] for row in self.values]

    def mean(self) -> float:
        shape: tuple[int, int] = self.shape()
        return self.sum() / (shape[0] * shape[1])

    def square(self) -> list[list[float]]:
        shape: tuple[int, int] = self.shape()
        return [[self.values[i][j] ** 2 for j in range(shape[1])] for i in range(shape[0])]


## Étape 1 : Dataset

La première étape consiste à récupérer les données de notre dataset. Pour ce faire, voici une classe permettant d'extraire les données (à l'initialisation de notre classe) de notre dataset (notre fichier CSV) en Python.

In [15]:
from csv import reader

class Dataset(object):
    target: Matrix
    features: Matrix

    def __init__(self, path: str):
        self.target = Matrix([])
        self.features = Matrix([])
        self.__read_dataset(path)

    def __read_dataset(self, path: str) -> None:
        try:
            with open(path, mode="r", newline="") as file:
                r = reader(file)
                next(r)
                for row in r:
                    self.target.values.append([float(row[1])])
                    self.features.values.append([float(row[0])])
        except FileNotFoundError:
            raise RuntimeError(f"The file {path} does not exist.")
        except Exception as e:
            raise RuntimeError(f"An error occurred while reading the CSV file: {e}")

Représentation visuel de notre target et de nos features (non normalisé) contenu dans notre dataset.

![Raw Data Visualisation](images/raw_data.png)

Par convention, on note `m` le nombre d'exemple que l'on a dans notre dataset.
On note `n` le nombre de features que possède notre dataset (c'est-à-dire le nombre de colonne hormis la colonne target).

![Dataset representation](images/dataset.png)

Dans notre situation, nous aurons donc un vecteur target (prix) de taille `(m, 1)` et une matrice features (km) de taille `(m, 1)` où `m` est égal à 24.

Dans notre situation, nous avons :

$$
\vec{y}=\begin{pmatrix}240000\cr139800\cr\ldots\cr61789\cr\end{pmatrix}
$$
$$
\vec{x}=\begin{pmatrix}3650\cr3800\cr\ldots\cr8290\cr\end{pmatrix}
$$

## Étape 2 : Modèle

À partir du dataset (et de la représentation graphique présente plus haut), on peux visualiser un nuage de points. Pour ce projet, nous allons utiliser un modèle linéaire, cependant il existe d'autres type de modèle disponible.

Pour notre modèle linéaire, nous avons donc la formule suivante :

$$
f(x)=ax+b
$$

Ce modèle a deux paramètres, le premier `a` et le second `b` (ce sont les coéfficients de notre polynome).
Il est important de noter que c'est nous qui décidons quel modèle doit utiliser la machine et c'est la machine qui doit apprendre les paramètres.

Notre modèle est une généralisation de l'ensemble des points que l'on a à disposition dans notre dataset.
Un bon modèle est un modèle qui nous donne les plus petites erreurs.

## Étape 3 : Fonction Coût

Ce modèle quand on l'utilise par rapport à notre dataset, il nous donne des erreurs. 

![Error model](images/model_error.png)

Quand on assemble toutes ces erreurs, ça nous donne ce que l'on appel la `fonction coût`.

## Étape 4 : Algorithme de minimisation

L'algorithme de minimisation est une stratégie qui cherche à trouver quels sont les paramètres de notre modèle qui minimise la fonction coût, c'est-à-dire qui minimise l'ensemble de nos erreurs.

Pour minimiser cette fonction coût, nous utiliserons l'algorithme de la déscente de gradient.